In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import simweights
import pickle
import os, sys
import numpy as np
import matplotlib as mat
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.colors as colors
import matplotlib.gridspec as gridspec
import pandas as pd
import tables
import h5py
import math
from scipy.stats import mstats
import matplotlib as mpl
import matplotlib.font_manager as font_manager
from fonts import *



In [3]:
sys.path.append("/data/user/tvaneede/GlobalFit/EventGenerator/performance")
from weights import *
from utils import *
from files import files
from selections import create_selections

In [4]:
# version and plotting path
version = "v0"
files = files[version]


In [5]:
# open the files
for flavor in files:
    files[flavor]['hdf_file'] = pd.HDFStore(files[flavor]['file_path'],'r')
    files[flavor]['variables'] = get_variables( files[flavor]['hdf_file'] )
    files[flavor] = Append_Weights( files[flavor] )


In [6]:
keys_to_merge = {
    "NuE" : ["NuE_E2", "NuE_E3"],
    "NuMu" : ["NuMu_E2", "NuMu_E3"],
    "NuTau" : ["NuTau_E2", "NuTau_E3"],
    "NuAll" : ['NuE', "NuMu", "NuTau"],
}

for new_key in keys_to_merge:
    files[new_key] = {}
    files[new_key]['variables'] = {}

    for key in keys_to_merge[new_key]:
        for variable in files[key]['variables']:
            if variable not in files[new_key]['variables']:
                files[new_key]['variables'][variable] = []
            files[new_key]['variables'][variable].append( pd.Series(files[key]['variables'][variable]) )
    
    for variable in files[new_key]['variables']:
        files[new_key]['variables'][variable] = pd.concat(files[new_key]['variables'][variable])


In [7]:
from selections import create_selections

In [8]:
selections = create_selections( files["NuAll"] )

In [14]:
# check energy range files, number of events, files
for simulation in files:
    energy = files[simulation]['variables']['PrimaryNeutrinoEnergy']
    recoenergy = files[simulation]['variables']['RecoETot']
    print(20*"-", simulation)
    print("PrimaryNeutrinoEnergy", "min", np.log10(min(energy)), "max", np.log10(max(energy)) )
    print("RecoETot", "min", np.log10(min(recoenergy)), "max", np.log10(max(recoenergy)) )
    # print("nfiles", files[simulation]['nfiles'])
    # print("evts", len(energy), "evts/file", len(energy)/files[simulation]['nfiles'])

-------------------- NuMu_E2
PrimaryNeutrinoEnergy min 4.268040943088046 max 5.999973812693128
RecoETot min 3.8086085072777722 max 6.647068883397538
-------------------- NuMu_E3
PrimaryNeutrinoEnergy min 6.000211037695408 max 7.9999995501711965
RecoETot min 3.566534095449937 max 8.999187485840867
-------------------- NuE_E2
PrimaryNeutrinoEnergy min 4.224270113376386 max 5.999822921626159
RecoETot min 3.193123539436236 max 6.663020154407304
-------------------- NuE_E3
PrimaryNeutrinoEnergy min 6.000617947429419 max 7.998799430206418
RecoETot min 3.0566047856565914 max 8.194089357422973
-------------------- NuTau_E2
PrimaryNeutrinoEnergy min 4.29318812463011 max 5.999883151520779
RecoETot min 3.002160254131146 max 6.999179427964016
-------------------- NuTau_E3
PrimaryNeutrinoEnergy min 6.000399423763156 max 7.999359159730375
RecoETot min 3.7922517247456686 max 8.133814957047576
-------------------- NuE
PrimaryNeutrinoEnergy min 4.224270113376386 max 7.998799430206418
RecoETot min 3.056

In [22]:
# check true double cascades

for selection_name in ["TrueDoubleCascade", "DoubleCascade_TrueDoubleCascade", "DoubleCascadeEvtGen_TrueDoubleCascade"]:

    selection = selections[selection_name]

    true_type = files["NuAll"]['variables']["PrimaryNeutrinoType"][selection]
    print(20*"-", selection_name)
    print( len(true_type), sum( true_type == -16.0 ), sum( true_type == 16.0 ) )
    print( sum( true_type == -16.0 ) + sum( true_type == 16.0 ) == len(true_type) )


-------------------- TrueDoubleCascade
9207 4494 4713
True
-------------------- DoubleCascade_TrueDoubleCascade
2181 1078 1103
True
-------------------- DoubleCascadeEvtGen_TrueDoubleCascade
2145 1057 1088
True
